In [66]:
## calculating GEDI summary stats from extracted dataframes

In [67]:
#install.packages("s3")
#install.packages("doParallel")
#install.packages("RItools")    

In [68]:
library("terra")
library("dplyr")
library("sf")
#library("s3")
library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#library("doParallel")
#library("RItools")

In [69]:
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
gediwk <- 24
#iso3 <- "Bpt"

In [70]:
iso3 <- "BaE"
gedi_paf_BaE <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_BaE)

iso3 <- "BaW"
gedi_paf_BaW <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_BaW)

iso3 <- "Bca"
gedi_paf_Bca <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_Bca)

iso3 <- "Bce"
gedi_paf_Bce <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_Bce)

iso3 <- "Bma"
gedi_paf_Bma <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_Bma)

iso3 <- "Bpp"
gedi_paf_Bpp <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_Bpp)

iso3 <- "Bpt"
gedi_paf_Bpt <- list.files(paste(f.path,"WDPA_extract/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = TRUE)
length(gedi_paf_Bpt)


[1] 240

[1] 417

[1] 53

[1] 335

[1] 615

[1] 31

[1] 26

In [71]:
gedi_paf_ALL <- c(gedi_paf_BaE, gedi_paf_BaW, gedi_paf_Bca,
                  gedi_paf_Bce, gedi_paf_Bma, gedi_paf_Bpp, gedi_paf_Bpt)
length(gedi_paf_ALL)
gedi_paf_ALL[1]

[1] 1717

[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaE_wk24/BaE_pa_101761_gedi_wk_24_conti_biome_TSMBF.RDS"

In [72]:
strsplit(strsplit(basename(gedi_paf_ALL[1]),".RDS")[[1]], "_")[[1]][9]

[1] "TSMBF"

In [73]:
gedi_paid_ALL <- c()
gedi_paBIOME_all <- c()
gedi_wwfBIOME_all <- c()

for(i in 1:length(gedi_paf_ALL)){

    this_paf <- gedi_paf_ALL[i]
    this_paid <- basename(this_paf) %>% readr::parse_number() %>% unique()
    gedi_paid_ALL <- c(gedi_paid_ALL, this_paid)
    #print(this_paid)
    this_paBIOME <- strsplit(basename(this_paf),"_")[[1]][1]
    gedi_paBIOME_all <- c(gedi_paBIOME_all, this_paBIOME)
    #print(this_paid)
    this_wwfBIOME <- strsplit(strsplit(basename(this_paf),".RDS")[[1]], "_")[[1]][9]
    gedi_wwfBIOME_all <- c(gedi_wwfBIOME_all, this_wwfBIOME)
    
}

print(length(gedi_paid_ALL))
print(length(unique(gedi_paid_ALL)))
print(length(gedi_paBIOME_all))
print(length(gedi_wwfBIOME_all))

[1] 1717
[1] 1621
[1] 1717
[1] 1717


In [36]:
gedi_BIOMESid_ALL <- cbind(gedi_paid_ALL, gedi_paBIOME_all, gedi_paBIOME_all, gedi_wwfBIOME_all)
colnames(gedi_BIOMESid_ALL) <- c("pa_id", "bbiome", "bbiome2", "wwfbiom")
head(gedi_BIOMESid_ALL)

pa_id,bbiome,bbiome2,wwfbiom
101761,BaE,BaE,TSMBF
10795,BaE,BaE,TSMBF
10801,BaE,BaE,TSGSS+TSMBF
10802,BaE,BaE,TSMBF
10803,BaE,BaE,TSMBF
115686,BaE,BaE,TSMBF


In [39]:
for(i in 1:nrow(gedi_BIOMESid_ALL)){
    if(gedi_BIOMESid_ALL[i,"bbiome"] == "BaE") { gedi_BIOMESid_ALL[i,"bbiome2"] <- "Baz" }
    if(gedi_BIOMESid_ALL[i,"bbiome"] == "BaW") { gedi_BIOMESid_ALL[i,"bbiome2"] <- "Baz" }    
}
head(gedi_BIOMESid_ALL)

pa_id,bbiome,bbiome2,wwfbiom
101761,BaE,Baz,TSMBF
10795,BaE,Baz,TSMBF
10801,BaE,Baz,TSGSS+TSMBF
10802,BaE,Baz,TSMBF
10803,BaE,Baz,TSMBF
115686,BaE,Baz,TSMBF


In [40]:
write.csv(gedi_BIOMESid_ALL, file="/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/allPAs_BRA_biomes.csv", row.names=FALSE)

In [ ]:
##

In [74]:
gedi_BIOMESid_ALL <- read.csv("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/allPAs_BRA_biomes2.csv")

In [75]:
unique(gedi_BIOMESid_ALL[which(duplicated(gedi_BIOMESid_ALL$pa_id)==TRUE),"pa_id"])
saveRDS(unique(gedi_BIOMESid_ALL[which(duplicated(gedi_BIOMESid_ALL$pa_id)==TRUE),"pa_id"]),
        file="/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/allPAs_BRA_biomes2_duplicatedWDPAIDs.RDS")

integer(0)

In [76]:
rownum <- c()
for(i in 1:length(gedi_paid_DUP)){
    rownum <- c(rownum, which(gedi_BIOMESid_ALL[,"pa_id"]==gedi_paid_DUP[i]))
}
rownum
head(gedi_BIOMESid_ALL[rownum,])

integer(0)

pa_id,bbiome,bbiome2,wwfbiom
<int>,<chr>,<chr>,<chr>


In [28]:
gedi_paid_DUP <- gedi_paid_ALL[which(duplicated(gedi_paid_ALL))]
length(gedi_paid_DUP)

gedi_paid_UNQ <- setdiff(gedi_paid_ALL, gedi_paid_DUP)
length(gedi_paid_UNQ)

[1] 96

[1] 1527

In [9]:
for(i in 1:length(gedi_paid_DUP)){
#print(i)
DUP_id <- gedi_paid_DUP[i]
#print(DUP_id)
DUP_paf <- gedi_paf_ALL[which(gedi_paid_ALL == DUP_id)]
#print(length(DUP_paf))
wwfbiom <- strsplit(strsplit(DUP_paf[1], split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    
#    if(file.exists(output_file)){
#        print(paste("csv file already exists for ",DUP_id,sep=""))
#    }    
#    else
        
    if(length(DUP_paf) == 2){
    
        pa_metrics1 <- readRDS(DUP_paf[1]) %>% unique()
        print(nrow(pa_metrics1))
        pa_metrics2 <- readRDS(DUP_paf[2]) %>% unique() 
        print(nrow(pa_metrics2))
        pa_metrics_ALL <- rbind(pa_metrics1, pa_metrics2)    
        print(nrow(pa_metrics_ALL))    
        wwfbiom <- strsplit(strsplit(DUP_paf[1], split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    write.csv(pa_metrics_ALL, file=paste(f.path,"/WDPA_extract/pa_stats_ALL/BRA_pa_",DUP_id,"_gedi_wk24_",wwfbiom,".csv",sep=""))
    }
    else
    
    if(length(DUP_paf) == 3){
        pa_metrics1 <- readRDS(DUP_paf[1]) %>% unique()
        print(nrow(pa_metrics1))
        pa_metrics2 <- readRDS(DUP_paf[2]) %>% unique() 
        print(nrow(pa_metrics2))
        pa_metrics3 <- readRDS(DUP_paf[3]) %>% unique()
        print(nrow(pa_metrics3))
        pa_metrics_ALL <- rbind(pa_metrics1, pa_metrics2, pa_metrics3)
        print(nrow(pa_metrics_ALL))
        wwfbiom <- strsplit(strsplit(DUP_paf[1], split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    write.csv(pa_metrics_ALL, file=paste(f.path,"/WDPA_extract/pa_stats_ALL/BRA_pa_",DUP_id,"_gedi_wk24_",wwfbiom,".csv",sep=""))
    }
    else
        
    if(length(DUP_paf) > 3){
        print(paste("there are more than 3 files with pa_id = ", DUP_id, sep=""))
    }
}

[1] 4417
[1] 84261
[1] 88678
[1] 1145739
[1] 277117
[1] 1422856
[1] 10221
[1] 696
[1] 3609
[1] 14526
[1] 580585
[1] 242470
[1] 823055
[1] 2875511
[1] 154
[1] 2875665
[1] 7904
[1] 225071
[1] 232975
[1] 109803
[1] 172181
[1] 281984
[1] 185405
[1] 121595
[1] 307000
[1] 404263
[1] 138346
[1] 542609
[1] 16228
[1] 1022379
[1] 1038607
[1] 29359
[1] 1161184
[1] 1190543
[1] 1600281
[1] 701396
[1] 2301677
[1] 53713
[1] 395530
[1] 449243
[1] 21579
[1] 164539
[1] 186118
[1] 15002
[1] 76231
[1] 91233
[1] 25153
[1] 80798
[1] 105951
[1] 111715
[1] 723027
[1] 834742
[1] 41341
[1] 568766
[1] 610107
[1] 25154
[1] 283866
[1] 309020
[1] 819852
[1] 7469
[1] 827321
[1] 2673597
[1] 2030438
[1] 4704035
[1] 1011815
[1] 2255
[1] 1014070
[1] 837354
[1] 758621
[1] 33467
[1] 1629442
[1] 11370
[1] 415525
[1] 426895
[1] 1212885
[1] 17640
[1] 1230525
[1] 4932
[1] 759456
[1] 764388
[1] 26820
[1] 568630
[1] 595450
[1] 37786
[1] 3625
[1] 41411
[1] 607442
[1] 10323
[1] 617765
[1] 135001
[1] 141812
[1] 276813
[1] 3294
[1]

In [10]:
length(gedi_paid_UNQ)

[1] 1527

In [20]:
which(gedi_paid_ALL == 351813)

[1] 97

In [17]:
#------------
#from_i <- 1
from_i <- which(gedi_paid_ALL == 67816)
#for(i in 1:length(gedi_paid_UNQ)){
for(i in from_i:length(gedi_paid_UNQ)){

    UNQ_id <- gedi_paid_UNQ[i]
    UNQ_paf <- gedi_paf_ALL[which(gedi_paid_ALL == UNQ_id)]
    wwfbiom <- strsplit(strsplit(UNQ_paf[1], split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    #print(UNQ_paf)

#    if(file.exists(paste(f.path,"/WDPA_extract/pa_stats_ALL/BRA_pa_",UNQ_id,"_gedi_wk24_",wwfbiom,".csv",sep=""))){
#        print(paste("csv file already exists for ",UNQ_id,sep=""))
#    } else {

    print(UNQ_id)
    pa_metrics <- readRDS(UNQ_paf) %>% unique()
    wwfbiom <- strsplit(strsplit(UNQ_paf, split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    write.csv(pa_metrics,
        file=paste(f.path,"/WDPA_extract/pa_stats_ALL/BRA_pa_",UNQ_id,"_gedi_wk24_",wwfbiom,".csv",sep=""))
#    }
}

[1] NA


ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'unique': invalid 'description' argument


In [19]:
print(length(list.files(paste(f.path,"/WDPA_extract/pa_stats_ALL/",sep=""))))

[1] 1621


In [26]:
# process error files
    UNQ_id <- 33920
    UNQ_paf <- gedi_paf_ALL[which(gedi_paid_ALL == UNQ_id)]
print(UNQ_paf)
    wwfbiom <- strsplit(strsplit(UNQ_paf[1], split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    print(UNQ_id)
    pa_metrics <- readRDS(UNQ_paf) %>% unique()
    wwfbiom <- strsplit(strsplit(UNQ_paf, split=c("_conti_biome_"))[[1]][2], split=".RDS")[[1]]
    write.csv(pa_metrics,
        file=paste(f.path,"/WDPA_extract/pa_stats_ALL/BRA_pa_",UNQ_id,"_gedi_wk24_",wwfbiom,".csv",sep=""))
#

[1] "/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_extract/BaW_wk24/BaW_pa_33920_gedi_wk_24_conti_biome_TSMBF.RDS"
[1] 33920


In [4]:
length(list.files(paste(f.path,"/WDPA_extract/pa_stats_ALL/",sep="")))

[1] 1621